# Worldwide Earthquake Events API - Gold Layer Processing

In [1]:
from pyspark.sql.functions import when, col, udf
from pyspark.sql.types import StringType
# ensure the below library is installed on your fabric environment
import reverse_geocoder as rg

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 5, Finished, Available, Finished)

In [3]:
df = spark.read.table("earthquake_events_silver")#.filter(col('time') > start_date)

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 7, Finished, Available, Finished)

In [9]:
coordinates = (34.2758333, -117.0211667)

rg.search(coordinates)[0].get('cc')

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 13, Finished, Available, Finished)

'US'

In [10]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    coordinates = (float(lat), float(lon))
    return rg.search(coordinates)[0].get('cc')

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 14, Finished, Available, Finished)

In [11]:
# registering the udfs so they can be used on spark dataframes
get_country_code_udf = udf(get_country_code, StringType())

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 15, Finished, Available, Finished)

In [12]:
# adding country_code and city attributes
df_with_location = \
                df.\
                    withColumn("country_code", get_country_code_udf(col("latitude"), col("longitude")))

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 16, Finished, Available, Finished)

In [13]:
display(df_with_location)

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 17, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cf4188a9-6ba5-4e08-ba90-d15b19d4a2d3)

In [14]:
# adding significance classification
df_with_location_sig_class = \
                            df_with_location.\
                                withColumn('sig_class', 
                                            when(col("sig") < 100, "Low").\
                                            when((col("sig") >= 100) & (col("sig") < 500), "Moderate").\
                                            otherwise("High")
                                            )

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 18, Finished, Available, Finished)

In [15]:
display(df_with_location_sig_class)

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 38e398f0-9f57-48de-b1d7-df030663d215)

In [16]:
# appending the data to the gold table
df_with_location_sig_class.write.mode('append').saveAsTable('earthquake_events_gold')

StatementMeta(, 2b55b82b-0044-43fb-b974-886c8c532d4d, 20, Finished, Available, Finished)